<a href="https://colab.research.google.com/github/CoryTee/JaccardDistanceTweets/blob/master/notebooks/jaccard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sys
import math
from typing import Dict, Any
import pandas as pd
import numpy as np


divider = '\t\t----------------------------------------------------------------'

"""
handle files using a callback method, prevents repetition

source:
https://stackoverflow.com/questions/3277503/how-to-read-a-file-line-by-line-into-a-list
"""

def _FileIO__file_handler(file_path, mode, callback = lambda f: None):
  f = open(file_path, mode)
  try:
    return callback(f)
  except Exception as e:
    raise IOError("Failed to %s file" % ["write to", "read from"][mode.lower() in "r rb r+".split(" ")])
  finally:
    f.close()


class FileIO:
  # return the contents of a file
  def read(file_path, mode = "r"):
    return __file_handler(file_path, mode, lambda rf: rf.read())

  # get the lines of a file
  def lines(file_path, mode = "r", filter_fn = lambda line: len(line) > 0):
    return [line for line in FileIO.read(file_path, mode).strip().split("\n") if filter_fn(line)]

  # create or update a file (NOTE: can also be used to replace a file's original content)
  def write(file_path, new_content, mode = "w"):
    return __file_handler(file_path, mode, lambda wf: wf.write(new_content))

  # delete a file (if it exists)
  def delete(file_path):
    return os.remove() if os.path.isfile(file_path) else None

def jaccard_dist(set_a, set_b):
  """
  Computes the Jaccard Distance of two sample sets (A and B) which measures 
  dissimilarity between them. It is defined as the difference of the sizes of 
  the union and the intersection of two sets divided by the size of the union 
  of the sets.
  
  JD(a, b) = 1 - (|a_intersect_b|/|a_union_b|)
  
  How to interpret the result:
    -Small if the tweets are similar
    -Large if the tweets are not similar
    -0 if the tweets have the same words (not counting duplicates or ordering)
    -1 if they are completely different (i.e. no overlapping words)
    
  http://en.wikipedia.org/wiki/Jaccard_index
  """
  
  a_union_b = len(set_a.union(set_b))
  a_intersect_b = len(set_a.intersection(set_b))
  
  return  1.0 - (a_intersect_b / a_union_b)
 
"""
Misc helper functions
"""
def list_to_set(lst):
  return set(lst)

def print_iteration_results(clusters):
  """
  A helper function for nicely formatting the results of each clusting iteration 
  and display the results
  """
  cluster_count = 1
  for keys, values in clusters.items():
    #print("Iteration #%s:\n" % (str(iteration_num + 1),))
    #iteration_num = iteration_num + 1
    #
    
    print("Cluster #%s: " % (str(cluster_count),), end='', flush=True)
    
    cluster_count = cluster_count + 1
    
    print("[", end='', flush=True)

    # Used to help format output
    centroid_count = 0
    for k in values:
      centroid_count = centroid_count + 1

      if centroid_count - 1 == 0 or (centroid_count -1) % 5 == 0:
        print("%s" % (str(k),), end='', flush=True)
      elif centroid_count == len(values):
        print("", end='\n\n', flush=True)
      elif centroid_count % 5 != 0 :
        print(", %s" % (str(k),), end='', flush=True)
      else:
        print(", %s," % (str(k),), end='\n', flush=True)
            
    
    print(divider, end='\n\n', flush=True)
    
    
def is_equal_nested_lists(nlist_1, nlist_2):
  """
  Determines if two nested lists are equivalent
  
  Used for determining convergence of this implementation of the K-Means 
  algorithm
  """
  for key in nlist_1.keys():
    if not is_equal_lists(nlist_1[key], nlist_2[key]):
      return False
  return True
  
def is_equal_lists(list_1, list_2):
  """
  Determines if two lists contain the same elements
  """
  set_1 = set(list_1)
  set_2 = set(list_2)
  
  return set_1 == set_2


def check_cluster_accuracy(cluster, nrows, k):
  """
  A sanity check I should have coded long ago to help verify the correctness of
  the K-Means implementation
  
  Basically checks if each iteration has the correct number of clusters(k) and 
  that each iteration is not erroneously missing or contain extra values
  """
  # Return false is wrong number of clusters
  num_clusters = len(cluster)
  if num_clusters != k:
    print("ERROR: INVALID NUMBER OF CLUSTERS: %s" % (str(num_clusters),))
    return False
  
  # Check to make sure no data was duplicated or lost
  count = 0
  for i in cluster:
    count = count + len(i)
    
  if nrows != count:
    print("ERROR: Invalid number of instances: %s" % (str(count),))
    return False
  return True

def preprocess(data_url):
  preprocessing_fun = ['P', 'r', 'e', 'p', 'r', 'o', 'c', 'e', 's', 's', 'i', 'n', 'g', ' ', 'd', 'a', 't', 
  'a', '.', '.',  '.',  '.',  '.',  '.',  '.',  '.', 'b', 'e',  '.',  '.',  '.', 'p', 'a', 't', 'i', 'e', 'n', 't', '.']
  tweets = pd.read_json(data_url, lines=True, orient='records')

  # Drop all columns except for the tweet text and the tweet id to save memory
  tweets = tweets[['text', 'id']]

  num_rows = tweets.shape[0]

  # Split tweet text data using spaces and save to new 'list' column before set
  # conversion then apply the set_from_list function to the new list 'column'
  tweets['list'] = tweets['text'].str.split()

  # Convert the lists of words for tweets into sets
  tweets['set'] = tweets['list'].apply(list_to_set)

  # Remove list column to free up memory
  tweets = tweets.drop(['list'], axis=1)

  # Create a new column of lists for Jaccard Distances to all other Tweets
  distances_list = []
  preprocess_fun_len = len(preprocessing_fun)
  fun_iterations = 0
  # Calculate Jaccard Distances between each tweet

  for i in range(0, num_rows):
    distances: Dict[str, float] = {}
    
    # Don't really need, but helps with debugging
    tweet_a_id = str(tweets.loc[[i], ['id']]['id'][i])
    set_a = tweets.loc[[i], ['set']]['set'][i]

    if fun_iterations >= preprocess_fun_len:
      print(preprocessing_fun[-1], end='', flush=True)
    else:
      print(preprocessing_fun[fun_iterations], end='', flush=True)
    fun_iterations = fun_iterations + 1

    for j in range(0, num_rows):
      if i != j:
        set_b = tweets.loc[[j], ['set']]['set'][j]

        # Get ID for 2nd Tweet for identification during K-Means
        tweet_b_id = str(tweets.loc[[j], ['id']]['id'][j])

        # Calculate Jaccard Distance between tweets and save with the ID of the 
        # 2nd tweet
        distances[tweet_b_id] = jaccard_dist(set_a, set_b)
      else:
        pass
    
    num_of_distances = len(distances) 
    
    if num_of_distances != num_rows-1:
      print('ERROR: Incorrect number of distances counted')
      print('Number of Instances: %s' % (str(num_of_distances)))
      print('Tweet ID: %s' % (tweet_a_id,))
      
    distances_list.append(distances)

  list_size = len(distances_list)
  if list_size != num_rows:
    print('ERROR: Incorrect number of distance dictionaries generated')
    print('Number of Dictionaries: %s' % (str(list_size)))
    print('Required: %s' % (str(num_rows),))

  print('')
  tweets = tweets.assign(distances=pd.Series(distances_list).values)
  return tweets

def read_seeds(seed_location):
  # Read in seeds from file
  centroid_ids = []
  file_ext_lines = FileIO.lines(seed_location)
  for i, line in enumerate(file_ext_lines):
    #Drop comma from end of line
    if line[-1:] is ",":
      centroid_ids.append(line[:-1])
    else:
      centroid_ids.append(line)

  return centroid_ids

def output_results(clusters, overall_sse, filename):
  
  with open(filename, 'w') as f:
    for key, cluster in clusters.items():
      f.write("%s : " % (str(key),))
      for i in cluster:
        f.write("%s, " % (str(i),))
      f.write("\n")
      
    f.write("%s" % str(round(overall_sse, 3)))

def k_means(seeds, tweets):
  centroids = seeds
  random = tweets.sample(25)['id']
  centroids = []
  for i in random:
    centroids.append(str(i))

  centroids = seeds
  clusters = cluster_assignment(centroids, tweets)
  

  converged = False
  num_iterations = 1
  print("Iteration: %s\n" % (str(num_iterations),))
  print_iteration_results(clusters)
  while not converged:

    num_iterations = num_iterations + 1
    print("Iteration: %s\n" % (str(num_iterations),))
    # Calculate new centroids for next clustering round
    new_centroids = calculate_new_centroids(clusters, tweets)
    if is_equal_lists(centroids, new_centroids):
      print('No change in centroids')
      
    # Assign to clusters clusters 
    new_clusters = cluster_assignment(new_centroids, tweets)
    
    if is_equal_nested_lists(clusters, new_clusters):
      print('\tClusters are the same so convergence', end='\n\n', flush=True)
      converged = True
      clusters = new_clusters
      centroids = new_centroids
      print_iteration_results(new_clusters)
      break
      
    
    clusters = new_clusters
    centroids = new_centroids
    print_iteration_results(new_clusters)
    
  return {'centroids' : centroids, 'clusters' : clusters}


def cluster_assignment(centroid_ids, tweets):
  """
  A function that accepts a list of strings correspoding to the IDs of tweets
  that are the centroids for the clusters and an already pre-processed pandas 
  dataframe of tweets
  """
  # The list of nodes at each cluster
  clusters = {}
  
  # Initialize nested lists to hold cluster member ID and add the centroid's ID
  i = 0
  for cur_id in centroid_ids:
    clusters[i] = [str(cur_id)]
    i = i+1
  
  for i in range(0, num_rows):
    current_id = str(tweets.loc[[i], ['id']]['id'][i])
    
    # Skip as we already added the centoid ID to cluster
    if current_id in centroid_ids:
      pass
    else:
      # Get dictionary of distances from this node to all others and their IDs
      distances = tweets[tweets['id'] == np.int64(current_id)]['distances'][i]

      dist_to_centroids = {k: distances[str(k)] for k in centroid_ids}
      
      closest_centroid = str(min(dist_to_centroids, key=dist_to_centroids.get))
            
      index = centroid_ids.index(closest_centroid)
      (clusters[index]).append(current_id)
      
  return clusters


def calculate_new_centroids(clusters, tweets):
  new_centroids = []
  
  for key, value in clusters.items():
    new_centroid = str(find_new_cluster_center(value, tweets))
    
    new_centroids.append(new_centroid)

  return new_centroids
  
  
def find_new_cluster_center(cluster_ids, tweets):
  distance_sums = {}
  
  for tweet_id in cluster_ids:
    distance_sums[tweet_id] = calc_dist_to_other_nodes(tweet_id, 
                                                            cluster_ids, tweets)
    
  return min(distance_sums, key=distance_sums.get)
  
  
def calc_dist_to_other_nodes(current_id, cluster_ids, tweets):
  # Get distance data for current tweet (row in tweets df) to all others
  distances = tweets.loc[tweets['id'] == np.int64(current_id)]['distances']
  
  # distances is a pandas.Serial object, with values returning a list, and in our
  # case there is only one entry
  distances = distances.values[0]
  
  dist_sum = 0.0
  
  for i in cluster_ids:
    if str(i) != current_id:
      dist_sum = dist_sum + distances[str(i)]

  return dist_sum


def compute_sse(clusters, centroid_ids, tweets):
  sse = 0.0
  
  for key, cluster in clusters.items():
    for i in cluster:
      # Ignore the centroid's data when we come to is
      if i in centroid_ids:
        pass
      else:
        distances = tweets[tweets['id'] == np.int64(i)]['distances']
        distances = distances.values[0]
        
        sse = sse + math.pow(float(distances[centroid_ids[key]]), 2)
        
  return sse

In [101]:
NUM_CLUSTERS = 25
seed_location = '/content/InitialSeeds.txt'
data_url = 'file://localhost/content/Tweets.json'
output_file = '/content/tweets-k-means-output.txt'

tweets = preprocess(data_url)
centroid_ids = read_seeds(seed_location)

Preprocessing data........be...patient.....................................................................................................................................................................................................................


In [102]:
results = k_means(centroid_ids, tweets)
centroids = results['centroids']
clusters = results['clusters']
overall_sse = compute_sse(clusters, centroids, tweets)
print('Sum of Squared Errors: %s' % (str(round(overall_sse, 3)),))

output_results(clusters, overall_sse, output_file)

Iteration: 1

Cluster #1: [323906397735641088, 323906397609791488, 323906397618196483, 323906397853073410, 323906397962121216,
323906398012461057, 323906398230544385, 323906398314438656, 323906398352195585, 323906398826164225,
323906398993932289, 323906399149109248, 323906399295926273, 323906399300100096, 323906656318676993,
323907087551836160, 323907771256938496, 323908455545049088, 323908795254312962

		----------------------------------------------------------------

Cluster #2: [323906483584655360, 323906485249789952, 323911610236293120, 323915000567697409, 323916051614138368,
323920146454425600, 323921510282702848, 323923559799996417

		----------------------------------------------------------------

Cluster #3: [323906657333682176, 323906650987692034, 323906651209994241

		----------------------------------------------------------------

Cluster #4: [323907258301939713, 323906398176030720, 323906567294562306, 323910330315075584, 323910330457669633,
323955716392112128, 3239639017